In [4]:
!pip install pyarrow --quiet 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
!ls

Untitled.ipynb	data


In [59]:
df = pd.read_parquet('data/fhv_tripdata_2021-01.parquet')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


In [5]:
df.shape

(1154112, 7)

In [6]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [67]:
df['dropOff_month'] = df['dropOff_datetime'].dt.month
df['pickup_month'] = df['pickup_datetime'].dt.month
df['duration'] = (df['dropOff_datetime'] - df['pickup_datetime']).dt.total_seconds()/60

In [71]:
f"The average trip duration in January is {df['duration'].mean()}"

'The average trip duration in January is 19.1672240937939'

In [74]:
f"The fractions of missing values for the pickup location ID is {null_df['PUlocationID']}"

'The fractions of missing values for the pickup location ID is 83.03067639882438'

In [97]:
# helper function
def prepare_data(df, train=True, model=None):
    print(f'Total records are {df.shape[0]}')
    
    df['dropOff_month'] = df['dropOff_datetime'].dt.month
    df['pickup_month'] = df['pickup_datetime'].dt.month
    df['duration'] = (df['dropOff_datetime'] - df['pickup_datetime']).dt.total_seconds()/60
    
    print(f"The average trip duration in January is {df['duration'].mean()}")
    
    duration_filter = (df['duration']>=1) & (df['duration'] <=60)
    print(f'Total records dropped are {df.shape[0] - df[duration_filter].shape[0]}')
    
    df = df[duration_filter].copy()
    
    null_df = (100 * df.isna().sum()) / df.shape[0]
    
    print(f"The fractions of missing values for the pickup location ID is {null_df['PUlocationID']}")
    
    df[['PUlocationID', 'DOlocationID']] = df[['PUlocationID', 'DOlocationID']].fillna(-1)
    
    categorical = ['PUlocationID', 'DOlocationID']
    numerical = ['duration']

    df[categorical] = df[categorical].astype(str)
    
    train_dicts = df[categorical ].to_dict(orient='records')
    
    if train:
        dv = DictVectorizer()
        dv_matrix = dv.fit_transform(train_dicts)
    
        print(f'The dimensionality of this matrix is {dv_matrix.shape[1]}')
        
        return dv, dv_matrix, df['duration'].values
    
    else:
        dv_matrix = model.transform(train_dicts)
        
        return dv_matrix, df['duration'].values

In [98]:
dvmodel, dv_matrix, y_train = prepare_data(df, train=True)

Total records are 1154112
The average trip duration in January is 19.1672240937939
Total records dropped are 44286
The fractions of missing values for the pickup location ID is 83.52732770722618
The dimensionality of this matrix is 525


In [99]:
lr = LinearRegression()
lr.fit(dv_matrix, y_train)

LinearRegression()

In [100]:
y_pred = lr.predict(dv_matrix)
mean_squared_error(y_train, y_pred, squared=False)

10.52851910722331

In [101]:
valid_data = pd.read_parquet('data/fhv_tripdata_2021-02.parquet')

In [103]:
valid_dv_matrix, y_valid = prepare_data(valid_data, train=False, model=dvmodel)

Total records are 1037692
The average trip duration in January is 20.70698622520125
Total records dropped are 47579
The fractions of missing values for the pickup location ID is 85.71354986754037


In [104]:
valid_pred = lr.predict(valid_dv_matrix)

In [105]:
mean_squared_error(valid_train, valid_pred, squared=False)

11.014283231616073